# 1- Load the required modules

In [1]:
import numpy as np                     # Llibreria matemÃ tica
import pandas as pd
import matplotlib.pyplot as plt        # Per mostrar plots
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds            # Per carregar mÃ©s facilment el dataset digits
import sklearn.model_selection as cv    # Pel Cross-validation
import sklearn.neighbors as nb         # Per fer servir el knn
%matplotlib inline 

# 2- Load the dataset

In [2]:
twigen = pd.read_csv("gender-classifier-DFE-791531.csv", encoding='latin1')
twigen.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


# Pre-processing

In [3]:
### Deleting not used properties
twigen = twigen.drop(['profile_yn', 'profile_yn:confidence', '_golden', '_unit_id', '_unit_state',
                     '_trusted_judgments', '_last_judgment_at', 'gender_gold', 'profile_yn_gold',
                     'profileimage', 'tweet_id', 'gender:confidence'], axis = 1)
print(twigen.columns)
### Deleting 'gender' -> 'unknown' rows

# print(len(twigen[twigen.gender == 'unknown']))
twigen = twigen.drop(twigen[twigen.gender == 'unknown'].index)
# print(len(twigen[twigen.gender == 'unknown']))

### Tuit creation time classification
from datetime import datetime
# print(twigen['tweet_created'])
twigen['hour_tweet_created'] = twigen['tweet_created'].apply(lambda t: datetime.strptime(t, '%m/%d/%y %H:%M').hour)
# print(twigen['hour_tweet_created'])

# Tuits are only created at midday (between 12h - 13h) probably the time when the tuits were obtained
def whichTimeRange(hour):
    if (hour in range(12, 14)): return 'midday'
    if (hour in range(1, 6)): return 'early_morning'
    if (hour in range(6, 12)): return 'morning'
    if (hour in range(14, 21)): return 'afternoon'
    if (hour in [21, 22, 23, 0]): return 'evening'
    return 'Unknown'

def markTimeRange(tuit):
    tuit[whichTimeRange(tuit.hour_tweet_created)] = 1
    return tuit

twigen['early_morning'] = twigen['morning'] = twigen['midday'] = twigen['afternoon'] = twigen['evening'] = 0
twigen = twigen.apply(markTimeRange, axis = 1)

print(twigen[['hour_tweet_created', 'early_morning', 'midday', 'afternoon', 'evening']])




Index(['gender', 'created', 'description', 'fav_number', 'link_color', 'name',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')
       hour_tweet_created  early_morning  midday  afternoon  evening
0                      12              0       1          0        0
1                      12              0       1          0        0
2                      12              0       1          0        0
3                      12              0       1          0        0
4                      12              0       1          0        0
5                      12              0       1          0        0
6                      12              0       1          0        0
7                      12              0       1          0        0
8                      12              0       1          0        0
9                      12              0       1          0        0
1

In [4]:
# Separate data from labels
labels = twigen['gender'] 
data = twigen.drop('gender', axis = 1)

# Data shape (n observations, n properties)
print(data.shape)
print(twigen.columns)

(18933, 19)
Index(['gender', 'created', 'description', 'fav_number', 'link_color', 'name',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_location', 'user_timezone',
       'hour_tweet_created', 'early_morning', 'morning', 'midday', 'afternoon',
       'evening'],
      dtype='object')


# 3- Data cross-validation

In [5]:
# Cross-validation: split data into training and test sets (test 30% of data)
(data_train, data_test, labels_train, labels_test) = cv.train_test_split(data, labels, test_size=.3, random_state=1)

# Create a kNN classifier object
knc = nb.KNeighborsClassifier()

# Train the classifier
knc.fit(data_train, labels_train)

# Obtain accuracy score of learned classifier on test data
#print(knc.score(data_test, labels_test))

ValueError: could not convert string to float: 'Eastern Time (US & Canada)'